# Evidently Tests and Test Presets

In [ ]:
import numpy as np

from sklearn import datasets
from sklearn import ensemble

from evidently.test_suite import TestSuite
from evidently.tests import *

from evidently.test_preset import NoTargetPerformanceTestPreset
from evidently.test_preset import DataQualityTestPreset
from evidently.test_preset import DataStabilityTestPreset
from evidently.test_preset import DataDriftTestPreset
from evidently.test_preset import RegressionTestPreset
from evidently.test_preset import MulticlassClassificationTestPreset
from evidently.test_preset import BinaryClassificationTopKTestPreset
from evidently.test_preset import BinaryClassificationTestPreset

## Prepare Datasets

In [ ]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan

In [ ]:
#Dataset for regression
housing_data = datasets.fetch_california_housing(as_frame='auto')
housing = housing_data.frame

housing.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing['prediction'] = housing_data['target'].values + np.random.normal(0, 3, housing.shape[0])

housing_ref = housing.sample(n=5000, replace=False)
housing_cur = housing.sample(n=5000, replace=False)

In [ ]:
#Dataset for binary probabilistic classifcation
bcancer_data = datasets.load_breast_cancer(as_frame='auto')
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['prediction'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['prediction'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]

In [ ]:
#Dataset for multiclass classifcation
iris_data = datasets.load_iris(as_frame='auto')
iris = iris_data.frame

iris_ref = iris.sample(n=75, replace=False)
iris_cur = iris.sample(n=75, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=3)
model.fit(iris_ref[iris_data.feature_names], iris_ref.target)

iris_ref['prediction'] = model.predict(iris_ref[iris_data.feature_names])
iris_cur['prediction'] = model.predict(iris_cur[iris_data.feature_names])

## How to run tests for a dataset?

In [ ]:
data_integrity_dataset_tests = TestSuite(tests=[
    TestNumberOfColumns(),
    TestNumberOfRows(),
    TestNumberOfMissingValues(),
    TestShareOfMissingValues(),
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestShareOfColumnsWithMissingValues(),
    TestShareOfRowsWithMissingValues(),
    TestNumberOfDifferentMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfEmptyRows(),
    TestNumberOfEmptyColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    
])

data_integrity_dataset_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_dataset_tests

In [ ]:
data_integrity_dataset_tests.json()

In [ ]:
data_integrity_dataset_tests.as_dict()

In [ ]:
data_quality_dataset_tests = TestSuite(tests=[
    TestTargetPredictionCorrelation(),
    TestHighlyCorrelatedColumns(),
    TestTargetFeaturesCorrelations(),
    TestPredictionFeaturesCorrelations(),
    TestCorrelationChanges(),
])

data_quality_dataset_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_quality_dataset_tests

In [ ]:
data_drift_dataset_tests = TestSuite(tests=[
    TestNumberOfDriftedColumns(),
    TestShareOfDriftedColumns(),
])

data_drift_dataset_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_dataset_tests

In [ ]:
regression_performance_dataset_tests = TestSuite(tests=[
    TestValueMAE(),
    TestValueRMSE(),
    TestValueMeanError(),
    TestValueMAPE(),
    TestValueAbsMaxError(),
    TestValueR2Score()
])

regression_performance_dataset_tests.run(reference_data=housing_ref, current_data=housing_cur)
regression_performance_dataset_tests

In [ ]:
classification_performance_dataset_tests = TestSuite(tests=[
    TestAccuracyScore(),
    TestPrecisionScore(),
    TestRecallScore(),
    TestF1Score(),
    TestPrecisionByClass(label='0'),
    TestPrecisionByClass(label='1'),
    TestPrecisionByClass(label='2'),
    TestRecallByClass(label='0'),
    TestRecallByClass(label='1'),
    TestRecallByClass(label='2'),
    TestF1ByClass(label='0'),
    TestF1ByClass(label='1'),
    TestF1ByClass(label='2'),
])

classification_performance_dataset_tests.run(reference_data=iris_ref, current_data=iris_cur)
classification_performance_dataset_tests

In [ ]:
prob_classification_performance_dataset_tests = TestSuite(tests=[
    TestAccuracyScore(),
    TestPrecisionScore(),
    TestRecallScore(),
    TestF1Score(),
    TestRocAuc(),
    TestLogLoss(),
    TestPrecisionByClass(label='0'),
    TestPrecisionByClass(label='1'),
    TestRecallByClass(label='0'),
    TestRecallByClass(label='1'),
    TestF1ByClass(label='0'),
    TestF1ByClass(label='1'),

])

prob_classification_performance_dataset_tests.run(reference_data=bcancer_ref, current_data=bcancer_cur)
prob_classification_performance_dataset_tests

## How to run tests for individual features?

In [ ]:
data_integrity_column_tests = TestSuite(tests=[
    TestColumnNumberOfMissingValues(column_name='education'),
    TestColumnShareOfMissingValues(column_name='education'),
    TestColumnNumberOfDifferentMissingValues(column_name='education'),
    TestColumnAllConstantValues(column_name='education'),
    TestColumnAllUniqueValues(column_name='education'),
    TestColumnValueRegExp(column_name='education',reg_exp='^[0..9]')
])

data_integrity_column_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_column_tests

In [ ]:
data_quality_column_tests = TestSuite(tests=[
    TestColumnValueMin(column_name='education-num'),
    TestColumnValueMax(column_name='education-num'),
    TestColumnValueMean(column_name='education-num'),
    TestColumnValueMedian(column_name='education-num'),
    TestColumnValueStd(column_name='education-num'),
    TestNumberOfUniqueValues(column_name='education'),
    TestUniqueValuesShare(column_name='education'),
    TestMostCommonValueShare(column_name='education'),
    TestMeanInNSigmas(column_name='education-num'),
    TestValueRange(column_name='education-num'),
    TestNumberOfOutRangeValues(column_name='education-num'),
    TestShareOfOutRangeValues(column_name='education-num'),
    TestValueList(column_name='education'),
    TestNumberOfOutListValues(column_name='education'),
    TestShareOfOutListValues(column_name='education'),
    TestValueQuantile(column_name='education-num', quantile=0.25),
    TestShareOfOutListValues(column_name='education-num'),
])

data_quality_column_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_quality_column_tests

In [ ]:
data_drift_column_tests = TestSuite(tests=[
    TestColumnDrift(column_name='education-num')
])

data_drift_column_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_column_tests

## How to set test parameters?

In [ ]:
feature_level_tests = TestSuite(tests=[
    TestMeanInNSigmas(column_name='hours-per-week', n_sigmas=3),
    TestShareOfOutRangeValues(column_name='hours-per-week', lte=0),
    TestColumnShareOfMissingValues(column_name='education', lt=0.2),
])

feature_level_tests.run(reference_data=adult_ref, current_data=adult_cur)
feature_level_tests

## How to use presets?

In [ ]:
no_target_performance = TestSuite(tests=[
    NoTargetPerformanceTestPreset(columns=['education-num', 'hours-per-week']),
])

no_target_performance.run(reference_data=adult_ref, current_data=adult_cur)
no_target_performance

In [ ]:
data_drift = TestSuite(tests=[
    DataDriftTestPreset(),
])

data_drift.run(reference_data=adult_ref, current_data=adult_cur)
data_drift

In [ ]:
data_stability = TestSuite(tests=[
    DataStabilityTestPreset(),
])

data_stability.run(reference_data=adult_ref, current_data=adult_cur)
data_stability

In [ ]:
data_quality = TestSuite(tests=[
    DataQualityTestPreset(),
])

data_quality.run(reference_data=adult_ref, current_data=adult_cur)
data_quality

In [ ]:
regression_performance = TestSuite(tests=[
    RegressionTestPreset()
])

regression_performance.run(reference_data=housing_ref, current_data=housing_cur)
regression_performance

In [ ]:
classification_performance = TestSuite(tests=[
   MulticlassClassificationTestPreset(prediction_type='labels')
])

classification_performance.run(reference_data=iris_ref, current_data=iris_cur)
classification_performance

In [ ]:
binary_classification_performance = TestSuite(tests=[
    BinaryClassificationTestPreset(prediction_type='probas'),
])

binary_classification_performance.run(reference_data=bcancer_ref, current_data=bcancer_cur)
binary_classification_performance

In [ ]:
binary_topK_classification_performance = TestSuite(tests=[
    BinaryClassificationTopKTestPreset(k=10),
])

binary_topK_classification_performance.run(reference_data=bcancer_ref, current_data=bcancer_cur)
binary_topK_classification_performance